# Study Experiment Design Generation

This notebook analyzes the characteristics of different space-filling experiment design generation techniques provide by raxpy for different input spaces.

In [ ]:
from typing import Optional, Annotated

import raxpy
import raxpy.spaces.root as s
import raxpy.spaces.dimensions as d
import raxpy.spaces.complexity as c
import raxpy.does.lhs as lhs_doe
import raxpy.does.random as random_doe
import raxpy.does.assess as assess

In [ ]:
strategies = [
    (lhs_doe.generate_design, "LHD-by-TreeTraversal",[]),
    (random_doe.generate_design, "Random",[]),
    (lhs_doe.generate_seperate_designs_by_full_subspace, "LHD-by-SubSpace",[]),
    (lhs_doe.generate_design_with_projection, "LHD-Projection", []),
]

In [ ]:
def generate_designs(strategies, space: s.InputSpace, number_of_designs: int = 10, number_of_points: int = 100):
    for _, strategy in enumerate(strategies):
        for _ in range(number_of_designs):
            doe = strategy[0](space, number_of_points)

            assessment = assess.assess_with_all_metrics(doe)
            strategy[2].append((doe,assessment))
            

In [ ]:
space = s.InputSpace(
    dimensions=[
        d.Float(id="x1", lb=0.0, ub=1.0, nullable=True),
        d.Float(id="x2", lb=0.0, ub=1.0, nullable=True),
        d.Float(id="x3", lb=0.0, ub=1.0, nullable=True),
    ]
)

By default when creating dimensions, the target portion of values in a design to be null is unspecified. Creating a design without specifying these values, results in the whole design to have parameters. The following code assigns these portitions using a complexity analysis hueristic.

In [ ]:
c.assign_null_portions(s.create_level_iterable(space.children))

In [ ]:
generate_designs(strategies, space, number_of_designs=10)

In [ ]:
import matplotlib.pyplot as plt


discrepancy_data = [[t[1].measurements[assess.METRIC_WEIGHTED_DISCREPANCY] for t in strategy[2] ] for strategy in strategies]
fig, axs = plt.subplots(nrows=1, ncols=1, figsize=(9, 4))

axs.violinplot(discrepancy_data,
                  showmeans=False,
                  showmedians=True)
axs.set_title('Weighted Discrepancies (smaller the better)')

plt.show()

In [ ]:
discrepancy_data = [[t[1].measurements[assess.METRIC_WHOLE_MIN_POINT_DISTANCE] for t in strategy[2] ] for strategy in strategies]
fig, axs = plt.subplots(nrows=1, ncols=1, figsize=(9, 4))

axs.violinplot(discrepancy_data,
                  showmeans=False,
                  showmedians=True)
axs.set_title('Minimum Interpoint Distances (larger the better)')

plt.show()

In [ ]:
discrepancy_data = [[t[1].measurements[assess.METRIC_WHOLE_MIN_PROJECTED_DISTANCE] for t in strategy[2] ] for strategy in strategies]
fig, axs = plt.subplots(nrows=1, ncols=1, figsize=(9, 4))
print(discrepancy_data)
axs.violinplot(discrepancy_data,
                  showmeans=False,
                  showmedians=True)
axs.set_title('Minimum Projected Distances (larger the better)')

plt.show()

In [ ]:
def get_sub_space_assessments(strategies, dim_list, metric=assess.METRIC_DISCREPANCY):
    results = []
    for strategy in strategies:
        design_results = []
        for replication_data in strategy[2]:
            assessment = replication_data[1].get_full_sub_design_assessment(dim_list)
            if assessment is not None:
                design_results.append(assessment.measurements[metric])
        results.append(design_results)

    return results

In [ ]:
def plot_sub_space_assessments(strategies, dim_list, metric=assess.METRIC_DISCREPANCY):
    dis_values = get_sub_space_assessments(strategies, dim_list, metric)

    discrepancy_data = [[t for t in dis_values[i] ] for i in range(len(strategies))]
    fig, axs = plt.subplots(nrows=1, ncols=1, figsize=(9, 4))

    axs.violinplot(discrepancy_data,
                    showmeans=False,
                    showmedians=True)
    axs.set_xticks(list(i+1 for i in range(len(strategies))))
    axs.set_xticklabels(list(strategy[1] for strategy in strategies))

    axs.set_title(f'Discrepancy for {", ".join(dim_list)} full-sub-design (smaller the better)')

    plt.show()
    pass

In [ ]:
plot_sub_space_assessments(strategies, ["x1","x2","x3"], assess.METRIC_DISCREPANCY)

In [ ]:
plot_sub_space_assessments(strategies, ["x1","x2"], assess.METRIC_DISCREPANCY)

In [ ]:
plot_sub_space_assessments(strategies, ["x2","x3"], assess.METRIC_DISCREPANCY)

In [ ]:
plot_sub_space_assessments(strategies, ["x1","x3"], assess.METRIC_DISCREPANCY)

In [ ]:
from scipy.stats.qmc import discrepancy

In [ ]:
discrepancy([[0.0],[0.5],[1.0]],method="WD")

In [ ]:
discrepancy([[0.0],[0.1],[1.0]],method="WD")

In [ ]:
discrepancy([[0.0],[0.001],[1.0]],method="WD")

In [ ]:
discrepancy([[0.0],[0.1],[0.05]],method="WD")

In [ ]:
discrepancy([[0.0],[0.1],[0.2],[0.3],[0.4],[0.5]],method="WD")

In [ ]:
discrepancy([[0.0],[0.19],[0.2],[0.3],[0.4],[0.5]],method="WD")

In [ ]:
discrepancy([[0.0],[0.19],[0.2],[0.21],[0.4],[0.5]],method="WD")

In [ ]:
discrepancy([[0.0],[0.19],[0.2],[0.21],[0.49],[0.5]],method="WD")

In [ ]:
discrepancy([[0.0],[0.01],[0.2],[0.21],[0.49],[0.5]],method="WD")

In [ ]:
help(discrepancy)

In [ ]:
discrepancy([[0.0],[0.19],[0.2],[0.21],[0.49],[0.5]], method="CD")

In [ ]:
discrepancy([[0.0],[0.1],[0.2],[0.3],[0.4],[0.5]], method="CD")